In [2]:
# ===== Reduced Order Modeling with NGSolve + pyMOR (POD-Galerkin) =====
import numpy as np
from ngsolve import *
from netgen.geom2d import unit_square

from pymor.algorithms.pod import pod
from pymor.vectorarrays.numpy import NumpyVectorSpace
import matplotlib.pyplot as plt


In [31]:
from netgen.occ import *
from ngsolve import Mesh

# ---------- Parameters ----------
L, W, H = 3.0, 1.0, 1.0
baffle_thk = 0.05
baffle_len = 0.35*W
baffle_x   = [0.7, 1.5, 2.3]
half_model = False
maxh       = 0.1

# ---------- Build main channel ----------
p0 = Pnt(0,0,0)
p1 = Pnt(L,W,H)
channel = Box(p0, p1)
channel.faces.name = "walls"

# Cut by symmetry plane if half_model
if half_model:
    sym_half = HalfSpace(Pnt(0, W/2, 0), Vec(0,1,0))
    channel = channel * sym_half

# ---------- Add baffles ----------
# First build full channel (no half cut yet)
channel = Box(Pnt(0,0,0), Pnt(L,W,H))
channel.faces.name = "walls"

fluid = channel
for i, x0 in enumerate(baffle_x, 1):
    x0min = x0 - 0.5*baffle_thk
    x0max = x0 + 0.5*baffle_thk
    zmin, zmax = 0, H

    if i == 2:
        # Middle baffle from TOP wall
        ymin, ymax = W - baffle_len, W
    else:
        # Side baffles from BOTTOM wall
        ymin, ymax = 0, baffle_len

    plate = Box(Pnt(x0min, ymin, zmin), Pnt(x0max, ymax, zmax))
    plate.faces.name = f"baffle{i}"
    fluid = fluid - plate

# Now apply half-model cut AFTER baffles are created
if half_model:
    sym_half = HalfSpace(Pnt(0, W/2, 0), Vec(0,1,0))
    fluid = fluid * sym_half

# ---------- Boundary conditions ----------
fluid.faces.Min(X).name = "inlet"
fluid.faces.Max(X).name = "outlet"

# ---------- Build OCC geometry and mesh ----------
geo = OCCGeometry(fluid)
m = geo.GenerateMesh(maxh=maxh)
mesh = Mesh(m)

print("Mesh elements:", mesh.ne)
print("Boundary names:", mesh.GetBoundaries())
from ngsolve.webgui import Draw   # for Jupyter/web view
# or from ngsolve import Draw     # for the Netgen GUI window

Draw(mesh)


Mesh elements: 15948
Boundary names: ('inlet', 'walls', 'walls', 'walls', 'walls', 'baffle1', 'baffle2', 'baffle2', 'baffle2', 'walls', 'outlet', 'walls', 'baffle3', 'baffle3', 'baffle3', 'walls', 'baffle1', 'baffle1')


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

BaseWebGuiScene